In [1]:
import numpy as np
import pandas as pd
import gensim
from gensim.corpora import Dictionary

### Files locations

In [2]:
FILE_USERS = "../../data/users_2020_2021.csv"
FILE_USERS_TWEETS = "../../data/tweets_2020_2021.csv"
FILE_RETWEETERS = "../../data/retweets.csv"
FILE_RETWEETERS_USERS = "../../data/retweets_users.csv"
POPULARITY = "../../data/popularity_v2.csv"

# Topic analysis
LDA_MODEL = "../../data/models/simple_lda.model"
LDA_MODEL_DIC = "../../data/models/simple_lda_dic"
IDF_LDA_MODEL = "../../data/models/tf_idf_lda.model"
IDF_LDA_DIC = "../../data/models/tf_idf_lda_dic"

### Load Datasets

In [3]:
users_df = pd.read_csv(filepath_or_buffer=FILE_USERS, sep=",")
users_tweets_df = pd.read_csv(filepath_or_buffer=FILE_USERS_TWEETS, sep=",")

# users_df = users_df[:100]
# users_tweets_df = users_tweets_df[:100]

/Users/migueloliveira/PycharmProjects/SocialInsight/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
users_df

,Unnamed: 0,user_id,name,username,followers,following,tweet_count,verified,created_at
0,0,1384617512197783554,𝐋𝐘𝐀ᵛᵏ,dilftaegguk,2292,2351,25535,False,2021-04-20 21:18:48+00:00
1,1,2268455894,ᄅᄅᄅ✨,RiqolivinLee,1258,718,35135,False,2013-12-30 06:51:41+00:00
2,2,782988768,kiki,kkyaraaa,591,346,20208,False,2012-08-26 18:54:03+00:00
3,3,804025242410549250,Book Plots,aplotbot,2,0,7677,False,2016-11-30 18:12:10+00:00
4,4,981788729130926080,Insightzclub,insightzclub,11,13,426,False,2018-04-05 07:00:48+00:00
...,...,...,...,...,...,...,...,...,...
1056257,1056257,718777835398307840,1040044,allpunkedup,7102,4895,48933,False,2016-04-09 12:29:25
1056258,1056258,900667362,1040045,WizzardOfTime,9558,660,34899,False,2012-10-23 22:04:21
1056259,1056259,335363076,1040046,grumpyalget,649,1261,4422,False,2011-07-14 15:31:53
1056260,1056260,146597135,1040047,flpwgr,287,611,9983,False,2010-05-21 21:22:51


In [5]:
users_tweets_df

,Unnamed: 0,tweet_id,text,user_id,timestamp,tweet_type,like_count,reply_count,retweet_count,quote_count,device,lang,topics_ids,topics,referenced_tweets
0,0,1473095874767060995,Closed🙏🙏 https://t.co/PAQyzOLZhS,1384617512197783554,2021-12-21 00:59:59+00:00,Original Tweet,0,0,0,0,Android,en,[],[],NaN
1,1,1473095874763005952,My feelings don’t be hurt just disappointed,2268455894,2021-12-21 00:59:59+00:00,Original Tweet,4,0,0,1,iPhone,en,[],[],NaN
2,2,1473095874762969090,Officially done with this semester 🥳🥳🥳🥳🥳🥳🥳🥳,782988768,2021-12-21 00:59:59+00:00,Original Tweet,3,0,0,0,iPhone,en,[],[],NaN
3,3,1473095874758598657,A personified wolf discovers that their boy is...,804025242410549250,2021-12-21 00:59:59+00:00,Original Tweet,0,0,0,0,Bot,en,[],[],NaN
4,4,1473095874754617344,As companies head into 2022 the future is hybr...,981788729130926080,2021-12-21 00:59:59+00:00,Original Tweet,0,0,0,0,NaN,en,[],[],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056257,1056257,1277361122836791296,Bowling for Soup announce new song https://t.c...,718777835398307840,2020-06-28 21:59:58,Original Tweet,4,0,2,0,NaN,en,[],[],NaN
1056258,1056258,1277361122832789504,Here’s the #Proposal💍👰🏼 stream from last night...,900667362,2020-06-28 21:59:58,Original Tweet,13,0,3,0,iPad,en,"['46', '47', '71']","['Brand Category', 'Brand', 'Video Game']",NaN
1056259,1056259,1277361122757271558,No one will miss you https://t.co/Oaw52bgtLH,335363076,2020-06-28 21:59:58,Original Tweet,0,0,0,0,iPhone,en,[],[],NaN
1056260,1056260,1277361122744680448,How to use a Kolsch glass :trollface: https://...,146597135,2020-06-28 21:59:58,Original Tweet,1,2,0,0,iPhone,en,[],[],NaN


### Sort tweets by timestamp

In [6]:
users_tweets_df = users_tweets_df.sort_values(by='timestamp', ascending=True)
users_tweets_df.head(5)

,Unnamed: 0,tweet_id,text,user_id,timestamp,tweet_type,like_count,reply_count,retweet_count,quote_count,device,lang,topics_ids,topics,referenced_tweets
794525,794525,1212176606699180032,IM READY FOR BTS,1078318888851513347,2020-01-01 00:59:58,Original Tweet,0,0,0,0,iPhone,en,[],[],NaN
794528,794528,1212176606678044674,I have over 50 cousins i found out about and t...,591385371,2020-01-01 00:59:58,Original Tweet,2,0,0,0,iPhone,en,[],[],NaN
794529,794529,1212176606677979136,What/who do you miss the most from your childh...,1103804603877871616,2020-01-01 00:59:58,Original Tweet,0,1,0,0,NaN,en,[],[],NaN
794530,794530,1212176606627815431,Where is that same 70' Degree weather that we ...,32706756,2020-01-01 00:59:58,Original Tweet,0,0,0,0,NaN,en,"['65', '66', '119', '65', '66']","['Interests and Hobbies Vertical', 'Interests ...",NaN
794531,794531,1212176606619480064,Lmao the way I screamed!!! 😂😂😂😂 https://t.co/1...,42065215,2020-01-01 00:59:58,Original Tweet,0,0,0,0,iPhone,en,[],[],NaN


### Delete duplicate users

In [7]:
users_df = users_df.drop_duplicates(subset=['user_id'], keep="first")
users_df

,Unnamed: 0,user_id,name,username,followers,following,tweet_count,verified,created_at
0,0,1384617512197783554,𝐋𝐘𝐀ᵛᵏ,dilftaegguk,2292,2351,25535,False,2021-04-20 21:18:48+00:00
1,1,2268455894,ᄅᄅᄅ✨,RiqolivinLee,1258,718,35135,False,2013-12-30 06:51:41+00:00
2,2,782988768,kiki,kkyaraaa,591,346,20208,False,2012-08-26 18:54:03+00:00
3,3,804025242410549250,Book Plots,aplotbot,2,0,7677,False,2016-11-30 18:12:10+00:00
4,4,981788729130926080,Insightzclub,insightzclub,11,13,426,False,2018-04-05 07:00:48+00:00
...,...,...,...,...,...,...,...,...,...
1056255,1056255,2679084948,1040042,BerlinMusicStat,936,266,7474,False,2014-07-25 08:53:43
1056256,1056256,17710576,1040043,diegosalazar,760,3090,6411,False,2008-11-28 15:31:04
1056259,1056259,335363076,1040046,grumpyalget,649,1261,4422,False,2011-07-14 15:31:53
1056260,1056260,146597135,1040047,flpwgr,287,611,9983,False,2010-05-21 21:22:51


### Calculate tweet reach

#### These values indicate the maximum possible of people that could have seen a given tweet

To do this, a sum of all the original tweet user's followers is made with the followers from all the users that retweeted/quoted the tweet

Note: this assumes that if a retweet is then shared by retweet again, those next audiences are not taken into account

In [8]:
t_retweeters = pd.read_csv(filepath_or_buffer=FILE_RETWEETERS_USERS, sep=",")
t_retweeters_tweets = pd.read_csv(filepath_or_buffer=FILE_RETWEETERS, sep=",")

In [9]:
t_retweeters_tweets.head()

,Unnamed: 0,tweet_id,text,user_id,timestamp,tweet_type,like_count,reply_count,retweet_count,quote_count,device,lang,topics_ids,topics,referenced_tweets
0,0,1353629005497458688,RT @DriaPustaka: “I have always imagined that ...,1191025207709974530,2021-01-25 09:01:18+00:00,Retweet Tweet,0,0,1,0,Bot,en,[],[],"[ReferencedTweet(id: 1353628732087361537, type..."
1,1,1351668710201819138,RT @DriaPustaka: “Take no heed of her.... She ...,975687478534340610,2021-01-19 23:11:47+00:00,Retweet Tweet,0,0,1,0,iPhone,en,[],[],"[ReferencedTweet(id: 1351499686033571840, type..."
2,2,1351175296083488768,RT @DriaPustaka: “I have always imagined that ...,1191025207709974530,2021-01-18 14:31:08+00:00,Retweet Tweet,0,0,1,0,Bot,en,[],[],"[ReferencedTweet(id: 1351175042260983813, type..."
3,3,1350669195399323648,RT @DriaPustaka: “Books are mirrors: you only ...,1318998016620154881,2021-01-17 05:00:05+00:00,Retweet Tweet,0,0,1,0,NaN,en,[],[],"[ReferencedTweet(id: 1350669173727260676, type..."
4,4,1350261759446114307,RT @DriaPustaka: “If you have enough book spac...,305408199,2021-01-16 02:01:04+00:00,Retweet Tweet,0,0,1,0,Android,en,[],[],"[ReferencedTweet(id: 1349989721372413952, type..."


In [10]:
def get_ref_tweet_id(ref_tweets):
    g = str(ref_tweets)
    if isinstance(g, str) and g != 'nan':
        return g.split()[1].replace(',', '')
    else: return None

In [11]:
t_retweeters_tweets['ref_tweed_id'] = t_retweeters_tweets['referenced_tweets'].apply(lambda ref_tweet: get_ref_tweet_id(ref_tweet))

In [12]:
def get_retweeters_reach(tweet_id, total_reach):
    retweets_users_ids = t_retweeters_tweets[t_retweeters_tweets['ref_tweed_id'] == tweet_id]['user_id'].tolist()
    for user_id in retweets_users_ids:
        total_reach += t_retweeters[t_retweeters['user_id'] == user_id]['followers'].sum()


In [13]:
def calculate_tweet_reach(tweet_id, user_id):
    total_reach = 0
    total_reach += users_df[users_df['user_id'] == user_id]['followers'].sum()
    get_retweeters_reach(tweet_id, total_reach)
    return total_reach

In [14]:
users_tweets_df['reach'] = users_tweets_df.apply(lambda tweet_row: calculate_tweet_reach(tweet_row['tweet_id'], tweet_row['user_id']), axis=1)
users_tweets_df.head(5)

,Unnamed: 0,tweet_id,text,user_id,timestamp,tweet_type,like_count,reply_count,retweet_count,quote_count,device,lang,topics_ids,topics,referenced_tweets,reach
794525,794525,1212176606699180032,IM READY FOR BTS,1078318888851513347,2020-01-01 00:59:58,Original Tweet,0,0,0,0,iPhone,en,[],[],NaN,100
794528,794528,1212176606678044674,I have over 50 cousins i found out about and t...,591385371,2020-01-01 00:59:58,Original Tweet,2,0,0,0,iPhone,en,[],[],NaN,366
794529,794529,1212176606677979136,What/who do you miss the most from your childh...,1103804603877871616,2020-01-01 00:59:58,Original Tweet,0,1,0,0,NaN,en,[],[],NaN,1131
794530,794530,1212176606627815431,Where is that same 70' Degree weather that we ...,32706756,2020-01-01 00:59:58,Original Tweet,0,0,0,0,NaN,en,"['65', '66', '119', '65', '66']","['Interests and Hobbies Vertical', 'Interests ...",NaN,145
794531,794531,1212176606619480064,Lmao the way I screamed!!! 😂😂😂😂 https://t.co/1...,42065215,2020-01-01 00:59:58,Original Tweet,0,0,0,0,iPhone,en,[],[],NaN,23168


### Topic analysis

#### Loading the previously trained topic analysis model and it's evaluations of each tweet text

In [15]:
users_tweets_df.head(5)

,Unnamed: 0,tweet_id,text,user_id,timestamp,tweet_type,like_count,reply_count,retweet_count,quote_count,device,lang,topics_ids,topics,referenced_tweets,reach
794525,794525,1212176606699180032,IM READY FOR BTS,1078318888851513347,2020-01-01 00:59:58,Original Tweet,0,0,0,0,iPhone,en,[],[],NaN,100
794528,794528,1212176606678044674,I have over 50 cousins i found out about and t...,591385371,2020-01-01 00:59:58,Original Tweet,2,0,0,0,iPhone,en,[],[],NaN,366
794529,794529,1212176606677979136,What/who do you miss the most from your childh...,1103804603877871616,2020-01-01 00:59:58,Original Tweet,0,1,0,0,NaN,en,[],[],NaN,1131
794530,794530,1212176606627815431,Where is that same 70' Degree weather that we ...,32706756,2020-01-01 00:59:58,Original Tweet,0,0,0,0,NaN,en,"['65', '66', '119', '65', '66']","['Interests and Hobbies Vertical', 'Interests ...",NaN,145
794531,794531,1212176606619480064,Lmao the way I screamed!!! 😂😂😂😂 https://t.co/1...,42065215,2020-01-01 00:59:58,Original Tweet,0,0,0,0,iPhone,en,[],[],NaN,23168


In [16]:
import json

def process_topic(topics:str):
    s = topics
    for i in range(topics.count('\'')):
        s = s.replace('\'', '"')
    t = json.loads(s)
    if len(t) > 0:
        return t[0]
    return None

In [17]:
users_tweets_df['topics'] = users_tweets_df['topics'].apply(lambda topics: process_topic(topics))
users_tweets_df['topics_ids'] = users_tweets_df['topics_ids'].apply(lambda topics: process_topic(topics))
users_tweets_df.head(5)

,Unnamed: 0,tweet_id,text,user_id,timestamp,tweet_type,like_count,reply_count,retweet_count,quote_count,device,lang,topics_ids,topics,referenced_tweets,reach
794525,794525,1212176606699180032,IM READY FOR BTS,1078318888851513347,2020-01-01 00:59:58,Original Tweet,0,0,0,0,iPhone,en,None,None,NaN,100
794528,794528,1212176606678044674,I have over 50 cousins i found out about and t...,591385371,2020-01-01 00:59:58,Original Tweet,2,0,0,0,iPhone,en,None,None,NaN,366
794529,794529,1212176606677979136,What/who do you miss the most from your childh...,1103804603877871616,2020-01-01 00:59:58,Original Tweet,0,1,0,0,NaN,en,None,None,NaN,1131
794530,794530,1212176606627815431,Where is that same 70' Degree weather that we ...,32706756,2020-01-01 00:59:58,Original Tweet,0,0,0,0,NaN,en,65,Interests and Hobbies Vertical,NaN,145
794531,794531,1212176606619480064,Lmao the way I screamed!!! 😂😂😂😂 https://t.co/1...,42065215,2020-01-01 00:59:58,Original Tweet,0,0,0,0,iPhone,en,None,None,NaN,23168


### Sentiment analysis

In [18]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def sentiment_scores(sentence, prints):
    if prints: print("\nSentence:", sentence)

    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)

    if sentiment_dict['compound'] >= 0.05:
        if prints: print("Positive")
        return "Positive"
    elif sentiment_dict['compound'] <= - 0.05:
        if prints: print("Positive")
        return "Negative"
    else:
        if prints: print("Neutral")
        return "Neutral"

In [19]:
users_tweets_df['sentiment'] = users_tweets_df['text'].apply(lambda tweet_text: sentiment_scores(tweet_text, False))

In [20]:
users_tweets_df.head(5)

,Unnamed: 0,tweet_id,text,user_id,timestamp,tweet_type,like_count,reply_count,retweet_count,quote_count,device,lang,topics_ids,topics,referenced_tweets,reach,sentiment
794525,794525,1212176606699180032,IM READY FOR BTS,1078318888851513347,2020-01-01 00:59:58,Original Tweet,0,0,0,0,iPhone,en,None,None,NaN,100,Positive
794528,794528,1212176606678044674,I have over 50 cousins i found out about and t...,591385371,2020-01-01 00:59:58,Original Tweet,2,0,0,0,iPhone,en,None,None,NaN,366,Neutral
794529,794529,1212176606677979136,What/who do you miss the most from your childh...,1103804603877871616,2020-01-01 00:59:58,Original Tweet,0,1,0,0,NaN,en,None,None,NaN,1131,Negative
794530,794530,1212176606627815431,Where is that same 70' Degree weather that we ...,32706756,2020-01-01 00:59:58,Original Tweet,0,0,0,0,NaN,en,65,Interests and Hobbies Vertical,NaN,145,Negative
794531,794531,1212176606619480064,Lmao the way I screamed!!! 😂😂😂😂 https://t.co/1...,42065215,2020-01-01 00:59:58,Original Tweet,0,0,0,0,iPhone,en,None,None,NaN,23168,Positive


### Tweet popularity

To define tweet popularity, each tweet with at least 1 retweet or 1 quote tweet was considered popular (1)
if the criteria wasn't met it was marked as unpopular (0)

In [21]:
def tweet_popularity_label(retweet_count, quote_count):
    if retweet_count > 0 or quote_count > 0:
        return 1
    else:
        return 0

In [22]:
users_tweets_df['popularity'] = users_tweets_df.apply(lambda row: tweet_popularity_label(row['retweet_count'], row['quote_count']), axis=1)

In [23]:
users_tweets_df['popularity'].value_counts()

0    853120
1    203142
Name: popularity, dtype: int64

In [24]:
popularity_df = pd.merge(users_tweets_df[['tweet_id', 'text', 'timestamp', 'user_id', 'like_count', 'retweet_count', 'quote_count', 'reply_count', 'reach', 'topics_ids', 'topics', 'sentiment', 'popularity']],
                   users_df[['user_id', 'followers', 'following', 'tweet_count', 'verified', 'created_at']],
                   on="user_id")
popularity_df.head(5)

,tweet_id,text,timestamp,user_id,like_count,retweet_count,quote_count,reply_count,reach,topics_ids,topics,sentiment,popularity,followers,following,tweet_count,verified,created_at
0,1212176606699180032,IM READY FOR BTS,2020-01-01 00:59:58,1078318888851513347,0,0,0,0,100,None,None,Positive,0,100,390,3961,False,2018-12-27 15:57:12
1,1212176606678044674,I have over 50 cousins i found out about and t...,2020-01-01 00:59:58,591385371,2,0,0,0,366,None,None,Neutral,0,366,134,34882,False,2012-05-26 22:28:16
2,1212176606677979136,What/who do you miss the most from your childh...,2020-01-01 00:59:58,1103804603877871616,0,0,0,1,1131,None,None,Negative,0,1131,714,2212,False,2019-03-07 23:48:20
3,1212538995021713410,What is your best advice for living a happy life?,2020-01-02 00:59:58,1103804603877871616,0,0,1,0,1131,None,None,Positive,1,1131,714,2212,False,2019-03-07 23:48:20
4,1212176606627815431,Where is that same 70' Degree weather that we ...,2020-01-01 00:59:58,32706756,0,0,0,0,145,65,Interests and Hobbies Vertical,Negative,0,145,293,2860,False,2009-04-18 01:22:32


### Combine the two datasets into the final one

For every tweet select the relevant columns and merge the corresponding information from the original user

In [25]:
popularity_df.to_csv(POPULARITY, sep=',', date_format='%Y-%m-%d %H:%M:%S')

In [26]:
popularity_df

,tweet_id,text,timestamp,user_id,like_count,retweet_count,quote_count,reply_count,reach,topics_ids,topics,sentiment,popularity,followers,following,tweet_count,verified,created_at
0,1212176606699180032,IM READY FOR BTS,2020-01-01 00:59:58,1078318888851513347,0,0,0,0,100,None,None,Positive,0,100,390,3961,False,2018-12-27 15:57:12
1,1212176606678044674,I have over 50 cousins i found out about and t...,2020-01-01 00:59:58,591385371,2,0,0,0,366,None,None,Neutral,0,366,134,34882,False,2012-05-26 22:28:16
2,1212176606677979136,What/who do you miss the most from your childh...,2020-01-01 00:59:58,1103804603877871616,0,0,0,1,1131,None,None,Negative,0,1131,714,2212,False,2019-03-07 23:48:20
3,1212538995021713410,What is your best advice for living a happy life?,2020-01-02 00:59:58,1103804603877871616,0,0,1,0,1131,None,None,Positive,1,1131,714,2212,False,2019-03-07 23:48:20
4,1212176606627815431,Where is that same 70' Degree weather that we ...,2020-01-01 00:59:58,32706756,0,0,0,0,145,65,Interests and Hobbies Vertical,Negative,0,145,293,2860,False,2009-04-18 01:22:32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056257,1477036841400520704,WE KEEP ALL THE PARTY IN THE ROOM ALL NIGHT,2021-12-31 21:59:59+00:00,1404228064062722051,0,0,0,0,181,None,None,Positive,0,181,482,5561,False,2021-06-14 00:04:08+00:00
1056258,1477036841429716992,wasn't the dispatch couple reveal a dud for 20...,2021-12-31 21:59:59+00:00,1214674663126646785,0,0,0,1,127,None,None,Negative,0,127,63,29566,False,2020-01-07 22:26:27+00:00
1056259,1477036841429782531,Jaymes joung - infinite 🎵,2021-12-31 21:59:59+00:00,283225511,0,0,0,0,1352,None,None,Neutral,0,1352,1914,34181,False,2011-04-16 21:15:42+00:00
1056260,1477036841429872644,2022 y’all gonna see a fucking monster.,2021-12-31 21:59:59+00:00,602195855,0,1,0,0,1082,None,None,Neutral,1,1082,1325,18474,False,2012-06-07 20:59:09+00:00
